# Anxiety Detection Work


In [135]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [136]:
from numpy import mean
from numpy import std
import numpy as np
np.random.seed(0)
import pandas as pd
from matplotlib import pyplot
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import confusion_matrix
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import seaborn as sns

In [137]:
data=loadmat('/content/drive/MyDrive/MGRU_main/data5sec.mat')
dataloaded = np.asarray(data['datamerge'])

In [138]:
dataloaded.shape

(9516, 201)

In [139]:
X=dataloaded[:,0:dataloaded.shape[1]-1]
print(X)
X.shape

[[9.76502062e-02 7.16745575e-01 2.46489374e-01 ... 5.94217008e+00
  5.92042131e+00 1.08640289e+01]
 [9.95908039e-02 4.51734388e-01 9.49580197e-02 ... 6.11465264e+00
  6.09274730e+00 1.09498824e+01]
 [1.86938366e-01 8.57855764e-01 1.58848125e-01 ... 6.06498573e+00
  5.90152046e+00 1.08757157e+01]
 ...
 [3.36091357e+00 1.43954172e+00 1.85661237e+00 ... 6.27022103e+00
  8.79549512e+00 8.38990271e+00]
 [2.53701348e+00 2.43600740e+00 1.70623581e+00 ... 9.17622102e+00
  6.53788382e+00 8.59781098e+00]
 [1.41724499e-04 9.25764211e-04 2.09689962e-03 ... 5.05164860e+00
  4.86910674e+00 1.76119704e+00]]


(9516, 200)

In [140]:
y=dataloaded[:,dataloaded.shape[1]-1]
print(y)
y.shape

[3. 3. 3. ... 0. 0. 0.]


(9516,)

In [141]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) #can change to random state=0 for fixed case

In [142]:
#feature scaling
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [143]:
print(X_train.shape), print(X_test.shape)
print(y_train.shape), print(y_test.shape)

(7612, 200)
(1904, 200)
(7612,)
(1904,)


(None, None)

In [144]:
from imblearn.over_sampling import SMOTE
smote= SMOTE(sampling_strategy='minority')
X_sm,y_sm= smote.fit_resample(X_train,y_train)
print(X_sm.shape, y_sm.shape)

(8431, 200) (8431,)


**KNN classifier**

In [145]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=10)
classifier.fit(X_sm, y_sm)
y_pred = classifier.predict(X_test)
print("Results using KNN classifier")
cm1 = confusion_matrix(y_test, y_pred)
print("confusion matrix \n",cm1)
cm=pd.crosstab(y_test,y_pred)
accuracy=(np.diag(cm).sum()/cm.sum().sum())*100
print(accuracy)
print(pd.DataFrame(classification_report(y_test,y_pred,output_dict=True)).T)
Kappa=sklearn.metrics.cohen_kappa_score(y_test,y_pred) 
print('Kappa=',Kappa)

Results using KNN classifier
confusion matrix 
 [[408  30  39 131]
 [ 88 176  60  67]
 [144  98 168  64]
 [108  47  16 260]]
53.15126050420168
              precision    recall  f1-score      support
0.0            0.545455  0.671053  0.601770   608.000000
1.0            0.501425  0.450128  0.474394   391.000000
2.0            0.593640  0.354430  0.443857   474.000000
3.0            0.498084  0.603248  0.545645   431.000000
accuracy       0.531513  0.531513  0.531513     0.531513
macro avg      0.534651  0.519715  0.516417  1904.000000
weighted avg   0.537685  0.531513  0.523595  1904.000000
Kappa= 0.3648745525074727


Decision **tree**

In [146]:
from sklearn.tree import DecisionTreeClassifier
clf_model = DecisionTreeClassifier(criterion="gini", random_state=42, max_depth=10, min_samples_leaf=5)   
clf_model.fit(X_sm,y_sm)
y_pred = clf_model.predict(X_test)
print("Results using Decision tree classifier")
cm1 = confusion_matrix(y_test, y_pred)
print("confusion matrix \n",cm1)
cm=pd.crosstab(y_test,y_pred)
accuracy=(np.diag(cm).sum()/cm.sum().sum())*100
print(accuracy)
print(pd.DataFrame(classification_report(y_test,y_pred,output_dict=True)).T)
Kappa=sklearn.metrics.cohen_kappa_score(y_test,y_pred) 
print('Kappa=',Kappa)

Results using Decision tree classifier
confusion matrix 
 [[483  24  30  71]
 [ 32 313  35  11]
 [ 31  46 384  13]
 [ 69  14   9 339]]
79.77941176470588
              precision    recall  f1-score      support
0.0            0.785366  0.794408  0.789861   608.000000
1.0            0.788413  0.800512  0.794416   391.000000
2.0            0.838428  0.810127  0.824034   474.000000
3.0            0.781106  0.786543  0.783815   431.000000
accuracy       0.797794  0.797794  0.797794     0.797794
macro avg      0.798328  0.797897  0.798032  1904.000000
weighted avg   0.798237  0.797794  0.797935  1904.000000
Kappa= 0.7276890906612113


LGBM

In [147]:
import lightgbm as lgb
model = lgb.LGBMClassifier(learning_rate=0.25,max_depth=3,random_state=42)
model.fit(X_sm,y_sm,eval_set=[(X_test,y_test),(X_sm,y_sm)],
          verbose=20,eval_metric='logloss')
y_pred = model.predict(X_test)
print(confusion_matrix(y_test,y_pred))

cm=pd.crosstab(y_test,y_pred)

accuracy=(np.diag(cm).sum()/cm.sum().sum())*100
print('Accuracy=',accuracy)
print(pd.DataFrame(classification_report(y_test,y_pred,output_dict=True)).T)
Kappa=sklearn.metrics.cohen_kappa_score(y_test,y_pred) 
print('Kappa=',Kappa)

/usr/local/lib/python3.9/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[20]	training's multi_logloss: 0.461813	valid_0's multi_logloss: 0.503359
[40]	training's multi_logloss: 0.291846	valid_0's multi_logloss: 0.368388
[60]	training's multi_logloss: 0.210227	valid_0's multi_logloss: 0.317444
[80]	training's multi_logloss: 0.15123	valid_0's multi_logloss: 0.281373
[100]	training's multi_logloss: 0.113344	valid_0's multi_logloss: 0.260787
[[553   1   5  49]
 [  5 362  16   8]
 [ 10  15 449   0]
 [ 45   5  10 371]]
Accuracy= 91.12394957983193
              precision    recall  f1-score      support
0.0            0.902121  0.909539  0.905815   608.000000
1.0            0.945170  0.925831  0.935401   391.000000
2.0            0.935417  0.947257  0.941300   474.000000
3.0            0.866822  0.860789  0.863795   431.000000
accuracy       0.911239  0.911239  0.911239     0.911239
macro avg      0.912382  0.910854  0.911578  1904.000000
weighted avg   0.911260  0.911239  0.911213  1904.000000
Kappa= 0.8804149442669034


# XGBoost classifier

In [148]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier

In [157]:
# Extreme gradient Boosting classifier 

xgb = XGBClassifier(max_depth =8,
                        min_child_weight = 1,
                        gamma = 0.7,
                        subsample = 0.8,
                        colsample_bytree = 1,
                        learning_rate = 0.1,
                        reg_alpha=0.05)

xgb.fit(X_sm, y_sm)
y_pred = xgb.predict(X_test)

xgb_accuracy = accuracy_score(y_pred, y_test) * 100

#print('\033[1m' +'Best parameters : '+ '\033[0m', cv.best_params_)
print('\033[1m' +'Accuracy : {:.2f}%'.format(xgb_accuracy) + '\033[0m')
print('\033[1m' +'Classification report : '+ '\033[0m\n', classification_report(y_test, y_pred))

cm = confusion_matrix(y_pred, y_test)
print("confusion matrix \n",cm)

Accuracy : 92.38%
Classification report : 
               precision    recall  f1-score   support

         0.0       0.92      0.91      0.91       608
         1.0       0.94      0.95      0.95       391
         2.0       0.96      0.96      0.96       474
         3.0       0.88      0.88      0.88       431

    accuracy                           0.92      1904
   macro avg       0.92      0.92      0.92      1904
weighted avg       0.92      0.92      0.92      1904

confusion matrix 
 [[555   3   8  40]
 [  3 371  13   7]
 [  3  13 453   4]
 [ 47   4   0 380]]


In [158]:
cm=pd.crosstab(y_test,y_pred)
accuracy=(np.diag(cm).sum()/cm.sum().sum())*100
print(accuracy)
print(pd.DataFrame(classification_report(y_test,y_pred,output_dict=True)).T)
Kappa=sklearn.metrics.cohen_kappa_score(y_test,y_pred) 
print('Kappa=',Kappa)

92.3844537815126
              precision    recall  f1-score      support
0.0            0.915842  0.912829  0.914333   608.000000
1.0            0.941624  0.948849  0.945223   391.000000
2.0            0.957717  0.955696  0.956705   474.000000
3.0            0.881671  0.881671  0.881671   431.000000
accuracy       0.923845  0.923845  0.923845     0.923845
macro avg      0.924213  0.924761  0.924483  1904.000000
weighted avg   0.923826  0.923845  0.923831  1904.000000
Kappa= 0.8974728818223445


Random **Forest**

In [151]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 100) 
clf.fit(X_sm, y_sm)
y_pred = clf.predict(X_test)
print("Results using Random forest classifier")
cm1 = confusion_matrix(y_test, y_pred)
print("confusion matrix \n",cm1)
cm=pd.crosstab(y_test,y_pred)
accuracy=(np.diag(cm).sum()/cm.sum().sum())*100
print(accuracy)
print(pd.DataFrame(classification_report(y_test,y_pred,output_dict=True)).T)
Kappa=sklearn.metrics.cohen_kappa_score(y_test,y_pred) 
print('Kappa=',Kappa)

Results using Random forest classifier
confusion matrix 
 [[550   2   4  52]
 [ 11 304  54  22]
 [ 15   6 452   1]
 [ 53   8  11 359]]
87.44747899159664
              precision    recall  f1-score      support
0.0            0.874404  0.904605  0.889248   608.000000
1.0            0.950000  0.777494  0.855134   391.000000
2.0            0.867562  0.953586  0.908543   474.000000
3.0            0.827189  0.832947  0.830058   431.000000
accuracy       0.874475  0.874475  0.874475     0.874475
macro avg      0.879789  0.867158  0.870746  1904.000000
weighted avg   0.877537  0.874475  0.873647  1904.000000
Kappa= 0.830434303055235


**Extra Tree classifier**

In [152]:
from sklearn.ensemble import ExtraTreesClassifier
# Building the model
extra_tree_forest = ExtraTreesClassifier(n_estimators = 100,
                                        criterion ='entropy', max_features = 200)  
# Training the model
extra_tree_forest.fit(X_sm, y_sm)
y_pred = extra_tree_forest.predict(X_test)
print("Results using Extra tree classifier")
cm1 = confusion_matrix(y_test, y_pred)
print("confusion matrix \n",cm1)
cm=pd.crosstab(y_test,y_pred)
accuracy=(np.diag(cm).sum()/cm.sum().sum())*100
print(accuracy)
print(pd.DataFrame(classification_report(y_test,y_pred,output_dict=True)).T)
Kappa=sklearn.metrics.cohen_kappa_score(y_test,y_pred) 
print('Kappa=',Kappa)

Results using Extra tree classifier
confusion matrix 
 [[552   1   6  49]
 [  3 346  34   8]
 [  5  12 456   1]
 [ 51   9   4 367]]
90.38865546218487
              precision    recall  f1-score      support
0.0            0.903437  0.907895  0.905660   608.000000
1.0            0.940217  0.884910  0.911726   391.000000
2.0            0.912000  0.962025  0.936345   474.000000
3.0            0.863529  0.851508  0.857477   431.000000
accuracy       0.903887  0.903887  0.903887     0.903887
macro avg      0.904796  0.901585  0.902802  1904.000000
weighted avg   0.904088  0.903887  0.903638  1904.000000
Kappa= 0.8704552425673829
